In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)

df_total_coste = pd.read_parquet(f'../data/raw/df_total_coste.parquet')
df_total_indicadores = pd.read_parquet(f'../data/raw/df_total_indicadores.parquet')


df_coste = pd.read_parquet(f'../data/processed/df_coste.parquet')
df_indicadores = pd.read_parquet(f'../data/processed/df_indicadores.parquet')

In [25]:
df_coste.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336690 entries, 0 to 17939
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   Provincia              336690 non-null  category
 1   Código Ente Principal  336690 non-null  category
 2   Tipo Ente Principal    336690 non-null  category
 3   Nombre Ente Principal  336690 non-null  category
 4   Descripción            336690 non-null  category
 5   coste_efectivo         336690 non-null  float64 
dtypes: category(5), float64(1)
memory usage: 8.5 MB


In [26]:
df_coste.nunique()

Provincia                   46
Código Ente Principal     5570
Tipo Ente Principal          7
Nombre Ente Principal     5554
Descripción                 43
coste_efectivo           70818
dtype: int64

In [27]:
df_indicadores.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582717 entries, 0 to 27677
Data columns (total 7 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   Provincia                       582717 non-null  category
 1   Código Ente Principal           582717 non-null  category
 2   Tipo Ente Principal             582717 non-null  category
 3   Nombre Ente Principal           582717 non-null  category
 4   Descripción                     582717 non-null  category
 5   Unidades físicas de referencia  582717 non-null  category
 6   Nº unidades                     582717 non-null  float64 
dtypes: category(6), float64(1)
memory usage: 14.5 MB


In [28]:
df_indicadores.nunique()

Provincia                            46
Código Ente Principal              5570
Tipo Ente Principal                   7
Nombre Ente Principal              5554
Descripción                          43
Unidades físicas de referencia       59
Nº unidades                       35382
dtype: int64

In [29]:
df_coste.columns

Index(['Provincia', 'Código Ente Principal', 'Tipo Ente Principal',
       'Nombre Ente Principal', 'Descripción', 'coste_efectivo'],
      dtype='object')

In [30]:
peso=df_coste.groupby(by='Tipo Ente Principal').sum()

In [31]:
df_coste['coste_efectivo'].sum()

461049090058.33984

In [32]:
peso.apply(lambda x: round(((x/df_coste['coste_efectivo'].sum())*100),2))

,coste_efectivo
Tipo Ente Principal,
Agrupación de municipios,0.00
Ayuntamiento,98.85
Comarca,0.05
Diputación/Consejo/Cabildo,0.64
Entidad Metropolitana,0.36
Entidad de ámbito territorial inferior al municipio,0.01
Mancomunidad,0.09


In [33]:
df_coste.loc[(df_coste['Descripción']=='Abastos, mercados, lonjas ')&(df_coste['Nombre Ente Principal']=='Huércal de Almería'),'coste_efectivo']=0

print(df_coste.loc[(df_coste['Descripción']=='Abastos, mercados, lonjas ')&(df_coste['Nombre Ente Principal']=='Huércal de Almería'),'coste_efectivo'])

df_coste.loc[(df_coste['Descripción']=='Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística')&(df_coste['Nombre Ente Principal']=='Rianxo'),'coste_efectivo']=78388.85 


df_coste.loc[(df_coste['Descripción']=='Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística')&(df_coste['Nombre Ente Principal']=='Rianxo'),'coste_efectivo']



1528    0.0
Name: coste_efectivo, dtype: float64


1932    78388.85
Name: coste_efectivo, dtype: float64

In [34]:
df_coste['Descripción'] = df_coste['Descripción'].astype('object')
df_indicadores['Descripción'] = df_indicadores['Descripción'].astype('object')


df_coste.loc[df_coste['Descripción']=='Abastecimiento domiciliario de agua potable','Descripción']='Abastecimiento de agua potable'
df_indicadores.loc[df_indicadores['Descripción']=='Abastecimiento domiciliario de agua potable','Descripción']='Abastecimiento de agua potable'

df_coste.loc[df_coste['Descripción']=='Conservación, mantenimiento y vigilancia de los edificios de titularidad local destinados a centros públicos de educación infantil, de educación primaria o de educación especial','Descripción']='Conservación, mantenimiento y vigilancia de los edificios destinados a educación'
df_indicadores.loc[df_indicadores['Descripción']=='Conservación, mantenimiento y vigilancia de los edificios de titularidad local destinados a centros públicos de educación infantil, de educación primaria o de educación especial','Descripción']='Conservación, mantenimiento y vigilancia de los edificios destinados a educación'


df_coste.loc[df_coste['Descripción']=='Cooperar con las Administraciones educativas correspondientes en la obtención de los solares necesarios para la construcción de nuevos centros docentes',
             'Descripción']='Cooperar con las Administraciones educativas en la obtención de solares'
df_indicadores.loc[df_indicadores['Descripción']=='Cooperar con las Administraciones educativas correspondientes en la obtención de los solares necesarios para la construcción de nuevos centros docentes',
                   'Descripción']='Cooperar con las Administraciones educativas en la obtención de solares'

df_coste.loc[df_coste['Descripción']=='Evaluación e información de situaciones de necesidad social y la atención inmediata a personas en situación o riesgo de exclusión social',
             'Descripción']='Necesidad social y atención inmediata a personas en riesgo de exclusión social'
df_indicadores.loc[df_indicadores['Descripción']=='Evaluación e información de situaciones de necesidad social y la atención inmediata a personas en situación o riesgo de exclusión social',
                   'Descripción']='Necesidad social y atención inmediata a personas en riesgo de exclusión social'

df_coste.loc[df_coste['Descripción']=='Medio ambiente urbano: Protección contra la contaminación acústica, lumínica y atmosférica en las zonas urbanas',
             'Descripción']='Protección contra contaminación acústica, lumínica y atmosférica'
df_indicadores.loc[df_indicadores['Descripción']=='Medio ambiente urbano: Protección contra la contaminación acústica, lumínica y atmosférica en las zonas urbanas',
                   'Descripción']='Protección contra contaminación acústica, lumínica y atmosférica'

df_coste.loc[df_coste['Descripción']=='Promoción en su término municipal de la participación de los ciudadanos en el uso eficiente y sostenible de las tecnologías de la información y las comunicaciones',
             'Descripción']='Promoción de la participación de los ciudadanos en las T.I.C.'
df_indicadores.loc[df_indicadores['Descripción']=='Promoción en su término municipal de la participación de los ciudadanos en el uso eficiente y sostenible de las tecnologías de la información y las comunicaciones',
                   'Descripción']='Promoción de la participación de los ciudadanos en las T.I.C.'

df_coste.loc[df_coste['Descripción']=='Promoción y gestión de la vivienda de protección pública con criterios de sostenibilidad financiera',
             'Descripción']='Promoción y gestión de la vivienda de protección pública'
df_indicadores.loc[df_indicadores['Descripción']=='Promoción y gestión de la vivienda de protección pública con criterios de sostenibilidad financiera',
                   'Descripción']='Promoción y gestión de la vivienda de protección pública'

df_coste['Descripción'] = df_coste['Descripción'].astype('category')
df_indicadores['Descripción'] = df_indicadores['Descripción'].astype('category')

print(df_coste['Descripción'].unique().to_list())
sorted(df_indicadores['Descripción'].unique().to_list())


['Alumbrado público', 'Cementerio', 'Recogida de residuos', 'Limpieza viaria', 'Abastecimiento de agua potable', 'Alcantarillado', 'Acceso a los núcleos de población', 'Pavimentación de las vías públicas', 'Parque público', 'Biblioteca pública', 'Tratamiento de residuos', 'Protección civil', 'Necesidad social y atención inmediata a personas en riesgo de exclusión social', 'Prevención y extinción de incendios', 'Instalaciones deportivas de uso público', 'Transporte colectivo urbano de viajeros', 'Medio ambiente urbano', 'Protección contra contaminación acústica, lumínica y atmosférica', 'Medio ambiente urbano: Parques y jardines públicos', 'Medio ambiente urbano: Gestión de los residuos sólidos urbanos', 'Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística', 'Protección y gestión del Patrimonio histórico', 'Promoción y gestión de la vivienda de protección pública', 'Conservación y rehabilitación de la edificación', 'Evacuación y tratamiento de aguas residuales', 'Infrae

['Abastecimiento de agua potable',
 'Abastos, mercados, lonjas ',
 'Acceso a los núcleos de población',
 'Actividades funerarias',
 'Alcantarillado',
 'Alumbrado público',
 'Biblioteca pública',
 'Cementerio',
 'Comercio ambulante',
 'Conservación y rehabilitación de la edificación',
 'Conservación, mantenimiento y vigilancia de los edificios destinados a educación',
 'Cooperar con las Administraciones educativas en la obtención de solares',
 'Equipamientos culturales',
 'Evacuación y tratamiento de aguas residuales',
 'Ferias ',
 'Información y promoción de la actividad turística de interés y ámbito local',
 'Infraestructura viaria y otros equipamientos de titularidad de la entidad local',
 'Instalaciones de ocupación del tiempo libre',
 'Instalaciones deportivas',
 'Instalaciones deportivas de uso público',
 'Limpieza viaria',
 'Medio ambiente urbano',
 'Medio ambiente urbano: Gestión de los residuos sólidos urbanos',
 'Medio ambiente urbano: Parques y jardines públicos',
 'Necesidad

In [35]:
df_coste['coste_efectivo'].sum()

33933999795.200005

In [36]:
peso=df_coste.groupby(by='Tipo Ente Principal').sum()

In [37]:
peso.apply(lambda x: round(((x/df_coste['coste_efectivo'].sum())*100),2))

,coste_efectivo
Tipo Ente Principal,
Agrupación de municipios,0.00
Ayuntamiento,84.40
Comarca,0.69
Diputación/Consejo/Cabildo,8.72
Entidad Metropolitana,4.83
Entidad de ámbito territorial inferior al municipio,0.08
Mancomunidad,1.27


In [38]:
# df_coste['cod_ente']=df_coste['Código Ente Principal']
# df_coste['cod_ente'] = df_coste['cod_ente'].str.replace(r'\d{2}-\d{2}-\d{3}-', '').str.replace(r'-\d{3}$', '')


In [39]:
# columns=['Provincia', 'Código Ente Principal', 'Tipo Ente Principal',
#        'Nombre Ente Principal', 'Descripción']

df_coste_pivot=df_coste.pivot_table(index=['Código Ente Principal'],values=['coste_efectivo'],columns=['Descripción'],aggfunc=np.sum)
df_coste_pivot.columns = df_coste_pivot.columns.droplevel()
df_coste_pivot.columns = df_coste_pivot.columns.tolist()
df_coste_pivot.columns
df_coste_pivot.reset_index(inplace=True)



In [40]:
more_columns=df_coste.groupby(by=['Código Ente Principal',])[['Provincia','Tipo Ente Principal','Nombre Ente Principal']].min().reset_index()

In [41]:
more_columns.nunique()

Código Ente Principal    5570
Provincia                  46
Tipo Ente Principal         7
Nombre Ente Principal    5554
dtype: int64

In [42]:
df_coste_pivot['Provincia']=more_columns['Provincia']
df_coste_pivot['Tipo Ente Principal']=more_columns['Tipo Ente Principal']
df_coste_pivot['Nombre Ente Principal']=more_columns['Nombre Ente Principal']






In [43]:
cols = df_coste_pivot.columns.tolist()
cols = cols[-3:] + cols[:-3] 


df_coste_pivot= df_coste_pivot[cols]

In [44]:
pd.set_option('display.max_columns', 10)

df_coste_pivot['codigoM']=df_coste_pivot['Código Ente Principal']

df_coste_pivot['codigoM'] = df_coste_pivot['codigoM'].str.replace(r'^.{3}|.{7}$|-', '')










In [45]:
df_coste_pivot['Tipo Ente Principal'].value_counts()

Ayuntamiento                                           4869
Mancomunidad                                            311
Entidad de ámbito territorial inferior al municipio     280
Comarca                                                  53
Diputación/Consejo/Cabildo                               47
Agrupación de municipios                                  7
Entidad Metropolitana                                     3
Name: Tipo Ente Principal, dtype: int64

In [46]:
df_final=df_coste_pivot.loc[(df_coste_pivot['Tipo Ente Principal']=='Ayuntamiento')|\
                      (df_coste_pivot['Tipo Ente Principal']=='Entidad de ámbito territorial inferior al municipio')].reset_index()




df_final=df_final.drop(columns=['index'])

df_final




,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,Abastecimiento de agua potable,...,Transporte colectivo urbano de viajeros,Tratamiento de residuos,"Tráfico, estacionamiento de vehículos y movilidad","Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",codigoM
0,Almería,Ayuntamiento,Abla,01-04-001-AA-000,282928.39,...,0.0,0.0,4260.16,2275.98,04001
1,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,77227.64,...,0.0,0.0,0.00,16674.59,04002
2,Almería,Ayuntamiento,Adra,01-04-003-AA-000,1255574.16,...,0.0,0.0,144618.30,125720.58,04003
3,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,60828.11,...,0.0,0.0,0.00,5637.62,04004
4,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,157266.48,...,0.0,0.0,0.00,0.00,04005
5,Almería,Ayuntamiento,Albox,01-04-006-AA-000,0.00,...,0.0,0.0,75471.49,259966.22,04006
6,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,61945.05,...,0.0,0.0,0.00,18720.14,04007
7,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,50007.27,...,0.0,0.0,0.00,41055.51,04008
8,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,36579.39,...,0.0,0.0,0.00,0.00,04009
...,...,...,...,...,...,...,...,...,...,...,...


## POBLACION

In [47]:
df_pob=pd.read_excel(f'/home/carpiero/IRONHACK/Proyecto FINAL/Datos municipios.xlsx', sheet_name=0,header=0)



In [48]:
df_pob=df_pob[['Población 2018',  'superficie',  'entidades singulares menores', ' ']]

In [49]:
df_pob['codigoM']=df_pob[' ']

In [50]:
df_pob['codigoM'] = df_pob['codigoM'].str.replace(r'\D', '')

df_pob.rename(columns={' ': 'Municipio'},inplace=True)





In [51]:




df_pob.info(memory_usage='deep')

# df_pob.to_parquet('../data/processed/df_pob.parquet')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8131 entries, 0 to 8130
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Población 2018                8128 non-null   object 
 1   superficie                    8116 non-null   float64
 2   entidades singulares menores  8116 non-null   float64
 3   Municipio                     8129 non-null   object 
 4   codigoM                       8129 non-null   object 
dtypes: float64(2), object(3)
memory usage: 1.5 MB


In [52]:
df_final_pob = pd.merge(df_final, df_pob, on='codigoM', how='left')

In [53]:
df_final_pob=df_final_pob.drop(columns=['superficie',
       'entidades singulares menores', 'Municipio'])





In [54]:
df_final_pob['TOTAL']=range(df_final_pob.shape[0])


df_final_pob['TOTAL']=df_final_pob.apply(lambda new: df_final_pob.iloc[new['TOTAL'],4:47].sum(), axis=1)






In [55]:
df_final_pob['TOTAL'].sum()

28668117226.0

In [56]:
df_final_pob[['Tipo Ente Principal','TOTAL']].groupby('Tipo Ente Principal').sum()

,TOTAL
Tipo Ente Principal,
Ayuntamiento,2.864129e+10
Entidad de ámbito territorial inferior al municipio,2.682528e+07


In [57]:
# fig=px.histogram(df_final_pob, x='Población 2018',nbins=400,template='seaborn',title='Age',barmode='group')
# fig.show()

In [58]:

df_final_pob=df_final_pob.loc[df_final_pob['Tipo Ente Principal']=='Ayuntamiento']

df_final_pob['TOTAL'].sum()

28641291941.309998

In [59]:
df_final_pob.nunique()

Provincia                                                                 46
Tipo Ente Principal                                                        1
Nombre Ente Principal                                                   4862
Código Ente Principal                                                   4869
Abastecimiento de agua potable                                          3744
Abastos, mercados, lonjas                                                623
Acceso a los núcleos de población                                       2102
Actividades funerarias                                                  1860
Alcantarillado                                                          2970
Alumbrado público                                                       4696
Biblioteca pública                                                      2177
Cementerio                                                              3056
                                                                        ... 

In [60]:
df_final_pob['Población 2018'] = df_final_pob['Población 2018'].astype('float64')

df_final_pob['Población 2018'].sum()



39376812.0

In [61]:
df_final_pob['CCAA']=df_final_pob['Código Ente Principal']

df_final_pob['CCAA']=df_final_pob['CCAA'].str.replace(r'.{14}$', '')



df_final_pob.loc[df_final_pob['CCAA']=='01','CCAA']='Andalucía'
df_final_pob.loc[df_final_pob['CCAA']=='02','CCAA']='Aragón'
df_final_pob.loc[df_final_pob['CCAA']=='03','CCAA']='Principado de Asturias'
df_final_pob.loc[df_final_pob['CCAA']=='04','CCAA']='Illes Balears'
df_final_pob.loc[df_final_pob['CCAA']=='05','CCAA']='Canarias'
df_final_pob.loc[df_final_pob['CCAA']=='06','CCAA']='Cantabria'
df_final_pob.loc[df_final_pob['CCAA']=='07','CCAA']='Castilla y León'
df_final_pob.loc[df_final_pob['CCAA']=='08','CCAA']='Castilla-La Mancha'
df_final_pob.loc[df_final_pob['CCAA']=='09','CCAA']='Cataluña'
df_final_pob.loc[df_final_pob['CCAA']=='10','CCAA']='Extremadura'
df_final_pob.loc[df_final_pob['CCAA']=='11','CCAA']='Galicia'
df_final_pob.loc[df_final_pob['CCAA']=='12','CCAA']='Comunidad de Madrid'
df_final_pob.loc[df_final_pob['CCAA']=='13','CCAA']='Región de Murcia'
df_final_pob.loc[df_final_pob['CCAA']=='16','CCAA']='La Rioja'
df_final_pob.loc[df_final_pob['CCAA']=='17','CCAA']='Comunitat Valenciana'





In [62]:
df_final_pob['CCAA'].value_counts()

Castilla y León           1138
Cataluña                   757
Andalucía                  655
Comunitat Valenciana       489
Aragón                     464
Extremadura                320
Castilla-La Mancha         271
Galicia                    246
Comunidad de Madrid        112
La Rioja                    94
Cantabria                   90
Canarias                    75
Principado de Asturias      65
Illes Balears               54
Región de Murcia            39
Name: CCAA, dtype: int64

In [63]:
# df_final_pob.info(memory_usage='deep')

cols = df_final_pob.columns.tolist()
cols = cols[-1:] + cols[:-1] 


df_final_pob= df_final_pob[cols]

df_final_pob

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,...,"Tráfico, estacionamiento de vehículos y movilidad","Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",codigoM,Población 2018,TOTAL
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,...,4260.16,2275.98,04001,1235.0,1260913.66
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,...,0.00,16674.59,04002,1150.0,477183.96
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,...,144618.30,125720.58,04003,24859.0,17946189.52
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,...,0.00,5637.62,04004,753.0,424815.71
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,...,0.00,0.00,04005,621.0,707330.39
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,...,75471.49,259966.22,04006,11696.0,5217127.03
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,...,0.00,18720.14,04007,814.0,743328.54
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,...,0.00,41055.51,04008,528.0,431192.73
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,...,0.00,0.00,04009,135.0,185199.72
...,...,...,...,...,...,...,...,...,...,...,...


In [64]:
cols = df_final_pob.columns.tolist()
cols = cols[5:48] + cols[50:51]


    
for x in cols:    
    df_final_pob[f'PC_{x}'] = df_final_pob.apply(lambda new: new[x]/new['Población 2018'], axis=1)



df_final_pob

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,...,PC_Transporte colectivo urbano de viajeros,PC_Tratamiento de residuos,"PC_Tráfico, estacionamiento de vehículos y movilidad","PC_Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",PC_TOTAL
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,...,0.0,0.000000,3.449522,1.842899,1020.982721
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,...,0.0,0.000000,0.000000,14.499643,414.942574
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,...,0.0,0.000000,5.817543,5.057347,721.919205
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,...,0.0,0.000000,0.000000,7.486879,564.164290
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,...,0.0,0.000000,0.000000,0.000000,1139.018341
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,...,0.0,0.000000,6.452761,22.226934,446.060793
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,...,0.0,0.000000,0.000000,22.997715,913.180025
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,...,0.0,0.000000,0.000000,77.756648,816.652898
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,...,0.0,0.000000,0.000000,0.000000,1371.849778
...,...,...,...,...,...,...,...,...,...,...,...


# Bidding

In [65]:
df_final_pob['cohorte_pob']=pd.cut(df_final_pob['Población 2018'], bins=[0,1000,2000,5000,10000,20000,50000,100000,200000,500000,6000000],
                                  labels=['0-1000','1000-2000','2000-5000','5000-10000','10000-20000',
                                          '20000-50000','50000-100000','100000-200000','200000-500000','500000-5000000'])

In [66]:
df_final_pob['cohorte_pob'].value_counts()

0-1000            2562
2000-5000          668
1000-2000          590
5000-10000         419
10000-20000        279
20000-50000        224
50000-100000        73
100000-200000       29
200000-500000       19
500000-5000000       6
Name: cohorte_pob, dtype: int64

In [67]:
df_final_pob

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,...,PC_Tratamiento de residuos,"PC_Tráfico, estacionamiento de vehículos y movilidad","PC_Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",PC_TOTAL,cohorte_pob
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,...,0.000000,3.449522,1.842899,1020.982721,1000-2000
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,...,0.000000,0.000000,14.499643,414.942574,1000-2000
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,...,0.000000,5.817543,5.057347,721.919205,20000-50000
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,...,0.000000,0.000000,7.486879,564.164290,0-1000
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,...,0.000000,0.000000,0.000000,1139.018341,0-1000
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,...,0.000000,6.452761,22.226934,446.060793,10000-20000
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,...,0.000000,0.000000,22.997715,913.180025,0-1000
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,...,0.000000,0.000000,77.756648,816.652898,0-1000
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,...,0.000000,0.000000,0.000000,1371.849778,0-1000
...,...,...,...,...,...,...,...,...,...,...,...


In [68]:

df_final_pob[df_final_pob.select_dtypes(['object']).columns] = \
            df_final_pob.select_dtypes(['object']).apply(lambda x: x.astype('category'))



df_final_pob.to_parquet('../data/processed/df_final_pob.parquet')



df_final_pob.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4869 entries, 0 to 5148
Data columns (total 96 columns):
 #   Column                                                                               Non-Null Count  Dtype   
---  ------                                                                               --------------  -----   
 0   CCAA                                                                                 4869 non-null   category
 1   Provincia                                                                            4869 non-null   category
 2   Tipo Ente Principal                                                                  4869 non-null   category
 3   Nombre Ente Principal                                                                4869 non-null   category
 4   Código Ente Principal                                                                4869 non-null   category
 5   Abastecimiento de agua potable                                                     

In [33]:
lo=df_coste.groupby(by=['Descripción','Nombre Ente Principal'],as_index=False).sum()

lo2=lo.loc[(lo['Descripción']=='Abastos, mercados, lonjas ')].sort_values(by='coste_efectivo',ascending=False)

In [34]:
lo2.to_excel(f'/home/carpiero/IRONHACK/zOld/lo.xlsx', index=False)

In [ ]:
df_indicadores.loc[(df_indicadores['Descripción']=='Actividades funerarias')&(df_indicadores['Código Ente Principal']=='17-46-904-AA-000')]